# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('weather.csv')
weather_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ushuaia,40,AR,1584309661,67,-54.80,-68.30,57.20,17.22
1,1,faanui,1,PF,1584309672,77,-16.48,-151.75,83.03,12.08
2,2,albany,20,US,1584309907,28,42.60,-73.97,45.00,12.75
3,3,renfrew,55,GB,1584309907,87,55.87,-4.39,45.00,4.70
4,4,victoria,20,HK,1584309907,68,22.29,114.16,68.00,11.41


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = weather_df[['Lat','Lng']]
weights = weather_df['Humidity']

In [4]:
humidity_map = gmaps.figure()
humidity_map.add_layer(gmaps.heatmap_layer(locations,weights=weights))
humidity_map

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Ideal weather is max temp between 70 and 80 degrees, wind speed less than 10 mph, and zero cloudiness
condition1 = weather_df['Max Temp']>=70
condition2 = weather_df['Max Temp']<=80
condition3 = weather_df['Wind Speed']<=10
condition4 = weather_df['Cloudiness'] == 0
narrowed_city_df = weather_df[(condition1&condition2&condition3&condition4)]
narrowed_city_df = narrowed_city_df.dropna()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = narrowed_city_df.copy()
hotel_df['Hotel Name'] = ""
hotel_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
144,144,samarai,0,PG,1584309667,85,-10.62,150.67,76.37,2.98,
152,152,shwebo,0,MM,1584309931,30,22.57,95.70,73.85,4.47,
173,173,marsh harbour,0,BS,1584309935,68,26.54,-77.06,73.31,9.24,
180,180,nsanje,0,MW,1584309936,92,-16.92,35.26,72.77,5.14,
236,236,dingle,0,PH,1584309732,87,11.00,122.67,70.79,8.43,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "lat,lng",
    "rankby": "distance",
    "keyword": "hotel",
    "key": g_key,
}

In [8]:
for index, row in hotel_df.iterrows():

    # get lat and lng from hotel_df
    city = row['City']
    coords = f"{row['Lat']},{row['Lng']}"

    # add keyword to params dict
    params['location'] = coords

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'Lat'] = results[0]['geometry']['location']['lat']
        hotel_df.loc[index, 'Lng'] = results[0]['geometry']['location']['lng']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Retrieving Results for Index 144: samarai.
------------
Retrieving Results for Index 152: shwebo.
------------
Retrieving Results for Index 173: marsh harbour.
------------
Retrieving Results for Index 180: nsanje.
------------
Retrieving Results for Index 236: dingle.
------------
Retrieving Results for Index 237: mount isa.
------------
Retrieving Results for Index 243: taoudenni.
Missing field/result... skipping.
------------
Retrieving Results for Index 269: kununurra.
------------
Retrieving Results for Index 408: iguape.
------------
Retrieving Results for Index 424: maningrida.
------------
Retrieving Results for Index 428: selu.
------------
Retrieving Results for Index 431: port hedland.
------------
Retrieving Results for Index 491: atar.
------------
Retrieving Results for Index 522: upington.
------------


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
hotel_df 

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
144,144,samarai,0,PG,1584309667,85,-10.625897,150.645083,76.37,2.98,Nuli Sapi
152,152,shwebo,0,MM,1584309931,30,22.568582,95.699824,73.85,4.47,MOON SHINE Guest House
173,173,marsh harbour,0,BS,1584309935,68,26.545827,-77.052221,73.31,9.24,Lofty Fig Villas
180,180,nsanje,0,MW,1584309936,92,-16.923563,35.257581,72.77,5.14,Nsanje Discovery Lodge
236,236,dingle,0,PH,1584309732,87,10.949491,122.636356,70.79,8.43,Oyo 236 Hotel Edmundo
237,237,mount isa,0,AU,1584309946,33,-20.730130,139.498581,73.40,5.82,Travellers Haven
243,243,taoudenni,0,ML,1584309947,7,22.680000,-3.980000,77.27,9.26,
269,269,kununurra,0,AU,1584309677,83,-15.770117,128.730738,71.60,3.02,Ivanhoe Village Caravan Resort
408,408,iguape,0,BR,1584309974,77,-24.709091,-47.558422,79.79,4.85,Pousada Casa Grande
424,424,maningrida,0,AU,1584309976,73,-12.056237,134.231798,79.97,5.03,Dhukurrdji Lodge


In [10]:
# Add marker layer ontop of heat map
hotel_map = gmaps.figure()
hotel_map.add_layer(gmaps.marker_layer(locations))
# Display Map
hotel_map

Figure(layout=FigureLayout(height='420px'))